In [21]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL

In [22]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tqdm

In [54]:
# this functions creates the dataset with the targets, given that the whole series is read
# i am not sure whether this is the right way for the training
def prepare_dataset(input_ds):
    targets = [label.numpy() for img, label in input_ds]
    target_digits = tf.convert_to_tensor(targets, dtype=tf.int32)



    #range_vals is only used to create the change between positive and negative signs for the summation
    range_vals = tf.range(len(targets))
    #this creates the alternating positve and negative signes by checkign whether the entry index modulo 2 is zero (i.e. even entries are positive, uneven ones negative)
    #check the tf.where documentation if the usage is confusing for you!
    alternating_target_numbers = tf.where(tf.math.floormod(range_vals,2)==0, target_digits, -target_digits)
    #finally we can compute the cumulative sums!
    c_sum = tf.math.cumsum(alternating_target_numbers)
    new_targets = tf.data.Dataset.from_tensor_slices(c_sum)

    ds = tf.data.Dataset.zip((input_ds, new_targets))

    # cast datatype to float
    ds = ds.map(lambda img, target: (tf.cast(img[0], tf.float32), target))

    # normalize values
    ds = ds.map(lambda img, target: (img/128. - 1., target))

    ds = ds.shuffle(1000) 
    ds = ds.batch(32)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    return ds

In [55]:
(train_dataset, test_dataset), ds_info = tfds.load(
    "mnist", split=["train", "test"], as_supervised=True, with_info=True
)

train_ds = prepare_dataset(train_dataset)
test_ds = prepare_dataset(test_dataset)

tf.Tensor([  4   3   3 ... -70 -69 -74], shape=(60000,), dtype=int32)
tf.Tensor([  2   2   6 ... 327 327 322], shape=(10000,), dtype=int32)


In [56]:
for img, label in train_ds.take(3):
    print(img.shape)
    print(label)

(32, 28, 28, 1)
tf.Tensor(
[ -40   19  -46 -133    4  -64  -17  -99    4   22   -5 -125  -57  -79
  -10   25   16 -108  -62 -105    6  -73  -55  -65 -124  -77   -4  -67
    3  -97  -17  -65], shape=(32,), dtype=int32)
(32, 28, 28, 1)
tf.Tensor(
[ -63  -52  -84  -16  -91   15  -60  -34  -78  -72    7  -65  -80    0
   -9  -78 -104  -40 -112  -40  -55 -118  -25  -55   -4  -75  -89  -13
  -41    0 -120  -60], shape=(32,), dtype=int32)
(32, 28, 28, 1)
tf.Tensor(
[   4   -8   17  -67  -12  -70  -75 -111  -63 -118  -35  -28    1  -83
  -67    5  -89   25  -62   18  -72  -61  -60  -63  -89  -92  -43  -32
   27  -82  -71  -62], shape=(32,), dtype=int32)
